In [1]:
import cv2
import numpy as np
import time
import glob
import math
import sys
import random
import os.path as osp
import pygame
import os
from socket import *



pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
host = "10.194.86.156" # set to IP address of target computer
port = 1234
addr = (host, port)
UDPSock = socket(AF_INET, SOCK_DGRAM)

In [ ]:
# Jinxin Function
#cap_top = cv2.VideoCapture(0)
USE_CAMERA = 1        
if USE_CAMERA:
    # USB camera setup
    cap_top = cv2.VideoCapture(1)
    if cap_top.isOpened() is False:
        raise("IO Error")
    cap_top.set(cv2.CAP_PROP_FPS, 30)
    cap_top.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap_top.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

windowWidth = 640
windowHeight = 480

#cap_top.set(cv2.CAP_PROP_FPS, 30)
area_lim2 = 650
hit_side_count = 0
p_side_time = time.time()
n_side_time = time.time()


pygame.init()
ka = pygame.mixer.Sound('Ka.WAV')

In [ ]:
while(1):
    
    global hit_side_count
    global n_side_time
    global p_side_time
    
    
    side_flag = None
    ret, frame = cap_top.read()
    cv2.imshow('frame', frame)
    data = '0'
    
    cv2.rectangle(frame,(170,170),(400,400),(0,255,0),0)            

    crop_image  = frame[170:400, 170:400]
    output = crop_image .copy()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)


    ret, frame = cap_top.read()
    crop_image  = frame[170:400, 170:400]
    frame_diff = cv2.absdiff(crop_image, output)
    gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame', frame)
    #cv2.circle(frame, (x, y), r, (0, 255, 0), 4)


    
    #par = np.int64(scaler*r)
    retval, thresh = cv2.threshold(gray_diff, 128, 255, cv2.THRESH_BINARY)
    #thresh_cut = cv2.resize(thresh[y-par:y+par,x-par:x+par],(mask_h,mask_w))

    cv2.imshow('frame4',thresh)

    #detect_stick_side(thresh_cut)
    _,cnt,hierarchy= cv2.findContours(thresh.astype('uint8'),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    area_list = []
    for i in range(len(cnt)):
        area_list.append(cv2.contourArea(cnt[i]))
    max_area = max(area_list, default=0)
    #print(max_area)
    if max_area> area_lim2:
        data = '1'
        #data = input('hit')
        ka.play()
        #data = '1'
    
    #data = input("Enter message to send or type 'exit': ")
    data = data.encode()
    UDPSock.sendto(data, addr)
    if data == "exit":
        break
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap_top.release()
cv2.destroyAllWindows()
UDPSock.close()
os._exit(0)